In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import random
import math

In [2]:
class Vertex:
    def __init__(self,label,x,y,blocked):
        self.xy=(x,y)
        self.label = self.xy
        self.color = 'white'
        self.dist = math.inf
        self.parent=None
        self.left=None
        self.right=None
        self.up=None
        self.down=None
        self.blocked=blocked #we store the node label here ('S'=start; 'D'=destination; '-'= open; 'X'= blocked node)

In [3]:
def find_vertex(x,y, graph):
    vt = Vertex('X',-1,-1,'X') #dummy vertex

    for v in graph:
        if graph[v].label==(x,y):
            if graph[v].blocked!='X':
                return graph[v]
            else:
                return vt

In [4]:
#helper function to populate graph with right, left,...
def build_graph(graph, rlen, clen):   

    for v in graph: #in graph
        # if y < (clen -1) assign right
        if graph[v].label[1]<(clen-1):
            if graph[v].blocked!='X':  # only going to find vertex if isn't blocked
                vt= find_vertex(graph[v].label[0], graph[v].label[1]+1, graph) #incrementing y
                if vt.label!=(-1,-1):
                    graph[v].right= vt #incrementing y
                    
        # if y > 0 assign left
        if graph[v].label[1]>0:
            if graph[v].blocked!='X': # only going to find vertex if isn't blocked
                vt= find_vertex(graph[v].label[0], graph[v].label[1]-1, graph) #decreasing y
                if vt.label!=(-1,-1):
                    graph[v].left= vt #decreasing y
                    
        #checking in the up for the entire grid, if x>0 assign up  
        if graph[v].label[0]>0:
            if graph[v].blocked!='X':  # only going to find vertex if isn't blocked
                vt= find_vertex(graph[v].label[0]-1, graph[v].label[1], graph) #decreasing x
                if vt.label!=(-1,-1):
                    graph[v].up= vt #decreasing x
                    
    #checking for down for entire grid, assign down except for last row, x<(rlen-1)
        if graph[v].label[0]<(rlen-1):
            if graph[v].blocked!='X':  # only going to find vertex if isn't blocked
                vt=graph[v].down= find_vertex(graph[v].label[0]+1, graph[v].label[1], graph) #increasing x 
                if vt.label!=(-1,-1):
                    graph[v].down=vt #increasing x
                    

In [5]:
def bfs(graph,s, d,alst):  
    
    #initialize graph
    for u in graph:
        graph[u].color = 'white'
        graph[u].dist = 0
        graph[u].parent = None
        
    graph[s].color = 'gray'
    graph[s].dist = 0
    graph[s].parent = None
    
    max_dist=0
    dfound=False
    Q=[]
    
    Q.append((graph[s]))
    
    while Q!=[]:
        coord=Q.pop(0) #dequeuing
        #if we found the destination, and visited all nodes at that level, then bfs is done
        if dfound and max_dist<=coord.dist:
            return alst
        
        # set the LEFT node
        if coord.left!=None and coord.left.label!=(-1,-1): 
            if coord.left.color == 'white':
                #set the node color,parent and dist
                graph[coord.left.label].color = 'gray' #make the graph's coord.left color 
                graph[coord.left.label].parent = coord
                graph[coord.left.label].dist = coord.dist + 1 
                # add the node to the queue
                Q.append(graph[coord.left.label])
                #add the node to the lst
                if len(alst)<(coord.dist+1): 
                    alst.append([coord.label,coord.left.label])
                else:
                    alst[coord.dist]=alst[coord.dist]+[coord.label,coord.left.label]

        #set the DOWN node
        if coord.down!=None and coord.down.label!=(-1,-1):
            if coord.down.color == 'white':
                graph[coord.down.label].color = 'gray'
                graph[coord.down.label].parent = coord
                graph[coord.down.label].dist =coord.dist + 1
                Q.append(graph[coord.down.label])
                if len(alst)<(coord.dist+1): 
                    alst.append([coord.label,coord.down.label])
                else:
                    alst[coord.dist]=alst[coord.dist]+[coord.label,coord.down.label]
                    
        # set the UP node
        if coord.up!=None and coord.up.label!=(-1,-1):
            if coord.up.color == 'white':
                graph[coord.up.label].color = 'gray'
                graph[coord.up.label].parent = coord
                graph[coord.up.label].dist = coord.dist + 1
                Q.append(graph[coord.up.label])
                if len(alst)<(coord.dist+1): 
                    alst.append([coord.label,coord.up.label])
                else:
                    alst[coord.dist]=alst[coord.dist]+[coord.label,coord.up.label]

        # set the RIGHT node
        if coord.right!=None and coord.right.label!=(-1,-1):
            if coord.right.color == 'white':
                graph[coord.right.label].color = 'gray'
                graph[coord.right.label].parent = coord
                graph[coord.right.label].dist = coord.dist + 1
                Q.append(graph[coord.right.label])
                if len(alst)<(coord.dist+1): 
                    alst.append([coord.label,coord.right.label])
                else:
                    alst[coord.dist]=alst[coord.dist]+[coord.label,coord.right.label]
                    
        #update max_dist
        max_dist=coord.dist+1
        #if destination reached set dfound to True
        if coord.label==d:
            dfound=True

        #this node has been visited now
        coord.color='black'
    return alst

In [6]:
def anim_bfs(grid): 
    global anmlst
    
    #intialize graph from grid
    graph = {}
    for xpt,row in enumerate(grid):
        for ypt,col in enumerate(row):
            graph[xpt,ypt] = Vertex(grid[xpt][ypt], xpt, ypt, grid[xpt][ypt])
            
    #find the starting and ending point coordinates
    for u in graph:
        if graph[u].blocked == 'S':
            spoints=graph[u].label
        if graph[u].blocked=='D':
            dpoints=graph[u].label
            
    # populate graph (right, left, up, down...)
    build_graph(graph, len(grid), (len(grid[0]))) 
    
    #BFS search for destination
    anmlst=[]
    anmlst=bfs(graph,spoints, dpoints,anmlst)
    
    #prepare for animation
    plt.rcParams["animation.html"] = "jshtml"
    plt.rcParams['figure.dpi'] = 150  
    plt.ioff()
    
    fig, ax = plt.subplots() 
    
    ax.set_xlim(-1,(len(grid[0])-1)+1, 1) # len(grid[0]
    ax.set_ylim((len(grid)-1)+1,-1, 1)
    
    ax.xaxis.set_ticks(np.arange(-1, (len(grid[0])-1)+1, 1))
    ax.yaxis.set_ticks(np.arange(-1,(len(grid)-1)+1, 1))

    plt.grid()
    
    #Plotting the points of the grid
    for xpt,row in enumerate(grid):
        for ypt,col in enumerate(row):
            x = xpt
            y = ypt
            if grid[x][y]=='X':
                plt.plot(y, x, marker="o", markersize=10,color='black')  
            elif grid[x][y]=='S':               
                plt.plot(y, x, marker="o", markersize=10,color='green')  
            elif grid[x][y]=='D':
                plt.plot(y, x, marker="o", markersize=10,color='red') 
                
    line =None
    max_dist=find_vertex(dpoints[0],dpoints[1],graph).dist
    col=['red','blue','green','brown','yellow','purple','orange']
    
    # animation function
    def animate_grid(t):
        #process the nodes with distance t
        lst_xy=anmlst[t]
        #cycle through list of colors
        c=col[t%(len(col))]
    
        for i in range(0,len(lst_xy),2):
            xpt1=lst_xy[i][1]
            ypt1=lst_xy[i][0]
            xpt2=lst_xy[i+1][1]
            ypt2=lst_xy[i+1][0]

            line, = ax.plot([xpt1,xpt2],[ypt1,ypt2], color=c ,lw=5) 

    return anim.FuncAnimation(fig, animate_grid, frames = np.arange(0,max_dist,1), interval=1000)

In [7]:
grid2 = [
'--------------------',
'-XX-X-XXX-----X-----',
'-XX---S-------X-----',
'-XXXX-XXX-----XXXX--',
'----X-XXX-----------',
'XXX-X-XXX-----X-----',
'X---X-XXX---XXXXXX-X',
'X-------------DX----',
'X-----X----X-XX-----'
]
anim_bfs(grid2)
